In [1]:
import json
import spacy
from spacy.tokens import DocBin

import re
import nltk
from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import pandas as pd

In [2]:
nltk.download('punkt')
nltk.download('punkt_tab')
# nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/vedanttibrewal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/vedanttibrewal/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vedanttibrewal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
lemmatizer = WordNetLemmatizer()

---------

In [63]:
stop_words = [
    'a', 'an', 'the',
    'i', 'me', 'my', 'mine', 'myself', 'you', 'your', 'yours', 'yourself', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'we', 'our', 'ours', 'ourselves', 'they', 'them', 'their', 'theirs', 'themselves',
    'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being',
    'do', 'does', 'did', 'have', 'has', 'had', 'can', 'could', 'shall', 'should', 'will', 'would', 'may', 'might', 'must',
    'about', 'across', 'after', 'against', 'along', 'around', 'at', 'before', 'behind', 'below', 'beneath', 'beside', 'during', 'into', 'near', 'off', 'out', 'through', 'toward', 'under', 'up', 'with',
    'if', 'then', 'because', 'as', 'until', 'while',
    'this', 'that', 'these', 'those', 'such', 'what', 'which', 'whose', 'whoever', 'whatever', 'whichever', 'whomever', 'either', 'neither', 'both',
    'very', 'really', 'always', 'never', 'too', 'already', 'often', 'sometimes', 'rarely', 'seldom', 'again', 'further', 'then', 'once', 'here', 'there', 'where', 'why', 'how',
    # 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'second', 'third', 'fourth', 'fifth', # find a way to implement it in limit
    'few', 'little', 'much', 'enough',
    'yes', 'no', 'not', 'okay', 'ok', 'right', 'sure', 'well', 'uh', 'um', 'oh', 'eh', 'hmm', 'just', 'ever', 'yet', 'etc', 'perhaps', 'maybe', 'list'
]

In [43]:


aggregate_functions = {
    'COUNT': ['count', 'number of', 'quantity of', 'total number of', 'tally', 'enumerate', 'how many'],
    'SUM': ['sum', 'total', 'aggregate', 'combined', 'add up', 'overall', 'cumulative'],
    'AVG': ['average', 'mean', 'typical', 'median', 'expected value', 'norm', 'central tendency'],
    'MAX': ['maximum', 'highest', 'most', 'top', 'peak', 'greatest', 'largest', 'biggest', 'uppermost'],
    'MIN': ['minimum', 'lowest', 'least', 'bottom', 'smallest', 'tiniest', 'least', 'floor'],
    'DISTINCT': ['unique', 'different', 'distinct', 'individual', 'separate', 'non-duplicate', 'exclusive'],
    'GROUP_CONCAT': ['concatenate', 'combine strings', 'join', 'merge text', 'string aggregation', 'text combination'],
    'FIRST': ['first', 'initial', 'earliest', 'primary', 'leading', 'opening', 'foremost'], # limit implementation along with number
    'LAST': ['last', 'final', 'latest', 'ultimate', 'concluding', 'terminal', 'closing']
}

comparison_operators = {
    '>': ['greater than', 'more than', 'above', 'over', 'exceeding', 'surpassing', 'beyond', 'higher than', 'in excess of'],
    '<': ['less than', 'fewer than', 'below', 'under', 'beneath', 'lower than', 'not as much as', 'smaller than'],
    '=': ['equal to', 'same as', 'identical to', 'matching', 'equivalent to', 'corresponds to', 'is', 'for'],
    '!=': ['not equal to', 'different from', 'excluding', 'not the same as', 'dissimilar to', 'unlike', 'other than'],
    '>=': ['greater than or equal to', 'at least', 'no less than', 'minimum of', 'not below', ' starting from'],
    '<=': ['less than or equal to', 'at most', 'no more than', 'maximum of', 'not above', 'up to'],
    'BETWEEN': ['between', 'in the range of', 'within the bounds of', 'inside the limits of'],
    'IN': ['in', 'within', 'among', 'included in', 'part of', 'contained in', 'one of'],
    'NOT IN': ['not in', 'outside of', 'excluded from', 'not among', 'not part of', 'not contained in'],
    'LIKE': ['like', 'similar to', 'resembling', 'matching pattern', 'corresponding to'],
    'NOT LIKE': ['not like', 'dissimilar to', 'unlike', 'not matching pattern', 'different from pattern']
}

logical_operators = {
    'AND': ['and', 'also', 'as well as', 'in addition to', 'plus', 'together with', 'along with', 'including'],
    'OR': ['or', 'alternatively', 'either', 'otherwise', 'else', 'and/or'],
    'NOT': ['not', 'except', 'excluding', 'other than', 'but not', 'save for', 'apart from']
}

In [6]:
data_types = {
    'INTEGER': ['integer', 'int', 'whole number', 'numeric'],
    'FLOAT': ['float', 'decimal', 'real number', 'fractional number'],
    'VARCHAR': ['string', 'text', 'characters', 'alphanumeric'],
    'DATE': ['date', 'calendar day', 'day'],
    'TIMESTAMP': ['timestamp', 'date and time', 'moment', 'point in time'],
    'BOOLEAN': ['boolean', 'true/false', 'yes/no', 'binary']
}

In [ ]:
# order matters
sql_clauses = {
    'FROM': ['from', 'in', 'out of', 'sourced from', 'derived from', 'based on'],
    'WHERE': ['where', 'for which', 'that have', 'meeting the condition', 'satisfying', 'fulfilling'],
    'GROUP BY': ['group by', 'categorize by', 'classify by', 'organize by', 'arrange by', 'cluster by', 'for each', 'broken down by', 'per', 'by'], # check about 'by'
    'HAVING': ['having', 'with the condition', 'subject to', 'meeting the criteria'],
    'ORDER BY': ['order by', 'sort by', 'arrange by', 'rank by', 'sequence by'],
    'LIMIT': ['limit', 'top', 'first', 'restrict to', 'cap at', 'only show'],
    'JOIN': ['join', 'combine', 'merge', 'connect', 'link', 'associate'],
    'UNION': ['union', 'combine', 'merge', 'incorporate', 'consolidate', 'unite'],
    'INTERSECT': ['intersect', 'in common', 'shared by', 'mutual', 'overlapping'],
    'EXCEPT': ['except', 'subtract', 'exclude', 'remove', 'omit', 'leave out'],
    'SELECT': ['show', 'list', 'give', 'return', 'fetch', 'retrieve', 'get', 'find', 'which', 'what is the', 'what is', 'what are', 'what'], # 'display
}

In [ ]:
# def replace_string(text, replace_dict):
#     # Create a reverse mapping from values to keys
#     value_to_key = {}
#     for key, values in replace_dict.items():
#         for value in values:
#             value_to_key[value.lower()] = key.lower()
    
#     # Split the text into words
#     words = text.split()
    
#     # Replace words
#     replaced_words = [value_to_key.get(word.lower(), word) for word in words]
    
#     return ' '.join(replaced_words)

def replace_string(text, replace_dict):
    # Create a reverse mapping from values to keys
    value_to_key = {}
    for key, values in replace_dict.items():
        for value in values:
            value_to_key[value.lower()] = key.lower()
    
    # Create a regex pattern for matching words
    pattern = r'\b(' + '|'.join(re.escape(word) for word in value_to_key.keys()) + r')\b'
    
    # Function to replace matched words
    def replace_word(match):
        return value_to_key.get(match.group(0).lower(), match.group(0))
    
    # Perform the replacement
    replaced_text = re.sub(pattern, replace_word, text, flags=re.IGNORECASE)
    
    return replaced_text


def preprocess_text(text):
    # Lowercase
    text = text.lower()
    
    # Remove punctuation
    # pattern = r'[^\w\s()*\d-]|(?<!\d)-(?!\d)'
    # text = re.sub(pattern, '', text)
    text = re.sub(r'\?', '', text)
    text = re.sub(r',', '', text)
    pattern = r'\b(\d+)-(\d+)\b'
    text = re.sub(pattern, r'\1 to \2', text)

    # substitute words
    text = replace_string(text, sql_clauses)
    text = replace_string(text, aggregate_functions)
    # from \d+ -> >=
    text = replace_string(text, comparison_operators)
    between_pattern = r'from \b(\w+|\d+)\b to \b(\w+|\d+)\b | \b(\w+|\d+)\b to \b(\w+|\d+)\b'
    text = re.sub(between_pattern, r" between(\1, \2)", text)
    text = replace_string(text, logical_operators)

    # Tokenization
    tokens = word_tokenize(text)

    # Remove stopwords
    #  load custom stop words
    tokens = [token for token in tokens if token not in stop_words]

    # Lemmatization
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)

In [64]:

def find_matches(text):
    # Regex pattern to match numbers, decimals, exponents, and quoted strings
    pattern = r"""
        (?<!\w)            # Negative lookbehind to ensure no word character before
        (                  # Start of group
            \d+\.\d+       # Decimal numbers
            (?:[eE][+-]?\d+)?  # Optional exponent part
            |              # OR
            \d+[eE][+-]?\d+  # Numbers with exponents
            |              # OR
            \d+            # Whole numbers
            |              # OR
            "(?:\\.|[^"\\])*"  # Double-quoted strings, allowing for escaped quotes
            |              # OR
            '(?:\\.|[^'\\])*'  # Single-quoted strings, allowing for escaped quotes
        )                  # End of group
        (?!\w)             # Negative lookahead to ensure no word character after
    """
    
    matches = re.findall(pattern, text, flags=re.VERBOSE)
    return matches

# Example usage
text = 'Here are some values: 42, 3.14, 2.71828e-1, "Hello, World!", and \'It\'s a test\'.'
result = find_matches(text)
print(result)

['42', '3.14', '2.71828e-1', '"Hello, World!"']


In [60]:
for sql in sql_clauses.values():
    # print(sql)
    for clause in sql:
        for func in aggregate_functions.values():
            for fn in func:
                if clause==fn:
                    print(clause, fn)

top top
first first
join join


---------

In [14]:
import re

def parse_sql_query(query):
    # Regular expressions to match SQL query components
    select_pattern = r'SELECT\s+(.*?)\s+FROM'
    from_pattern = r'FROM\s+(\w+)'
    join_pattern = r'JOIN\s+(\w+)\s+ON\s+([\w.]+)\s*=\s*([\w.]+)'
    where_pattern = r'WHERE\s+(.*?)(?:\s+GROUP BY|\s+ORDER BY|\s+LIMIT|$)'
    group_by_pattern = r'GROUP BY\s+(.*?)(?:\s+HAVING|\s+ORDER BY|\s+LIMIT|$)'
    having_pattern = r'HAVING\s+(.*?)(?:\s+ORDER BY|\s+LIMIT|$)'
    order_by_pattern = r'ORDER BY\s+(.*?)(?:\s+LIMIT|$)'
    limit_pattern = r'LIMIT\s+(\d+)'

    # Find matches based on the patterns
    select_match = re.search(select_pattern, query, re.IGNORECASE)
    from_match = re.search(from_pattern, query, re.IGNORECASE)
    join_match = re.search(join_pattern, query, re.IGNORECASE)
    where_match = re.search(where_pattern, query, re.IGNORECASE)
    group_by_match = re.search(group_by_pattern, query, re.IGNORECASE)
    having_match = re.search(having_pattern, query, re.IGNORECASE)
    order_by_match = re.search(order_by_pattern, query, re.IGNORECASE)
    limit_match = re.search(limit_pattern, query, re.IGNORECASE)

    result = {}

    if select_match:
        result['columns'] = [col.strip() for col in select_match.group(1).split(',')]

    if from_match:
        result['table'] = from_match.group(1)

    if join_match:
        result['join'] = {
            'table': join_match.group(1),
            'condition': f"{join_match.group(2)} = {join_match.group(3)}"
        }

    if where_match:
        result['where'] = where_match.group(1)

    if group_by_match:
        result['group_by'] = [col.strip() for col in group_by_match.group(1).split(',')]

    if having_match:
        result['having'] = having_match.group(1)

    if order_by_match:
        result['order_by'] = [col.strip() for col in order_by_match.group(1).split(',')]

    if limit_match:
        result['limit'] = int(limit_match.group(1))

    return result

# Test the function
query = "SELECT col1, col2 FROM table1 JOIN table2 ON table1.key = table2.key WHERE cond1 = 'value' GROUP BY col1 HAVING COUNT(*) > 5 ORDER BY col2 DESC LIMIT 10"
result = parse_sql_query(query)
print(f"Query: {query}")
print(f"Result: {result}")

Query: SELECT col1, col2 FROM table1 JOIN table2 ON table1.key = table2.key WHERE cond1 = 'value' GROUP BY col1 HAVING COUNT(*) > 5 ORDER BY col2 DESC LIMIT 10
Result: {'columns': ['col1', 'col2'], 'table': 'table1', 'join': {'table': 'table2', 'condition': 'table1.key = table2.key'}, 'where': "cond1 = 'value'", 'group_by': ['col1'], 'having': 'COUNT(*) > 5', 'order_by': ['col2 DESC'], 'limit': 10}


In [15]:
query = """select city_name from city where population  =  ( select max ( population ) from city where state_name  =  "wyoming" ) and state_name  =  "wyoming";"""
result = parse_sql_query(query)
print(f"Query: {query}")
print(f"Result: {result}")

Query: select city_name from city where population  =  ( select max ( population ) from city where state_name  =  "wyoming" ) and state_name  =  "wyoming";
Result: {'columns': ['city_name'], 'table': 'city', 'where': 'population  =  ( select max ( population ) from city where state_name  =  "wyoming" ) and state_name  =  "wyoming";'}


In [17]:
query = """select distinct t1.paperid from venue as t2 join paper as t1 on t2.venueid  =  t1.venueid where t2.venuename  =  "chi";"""

result = parse_sql_query(query)
print(f"Query: {query}")
print(f"Result: {result}")

Query: select distinct t1.paperid from venue as t2 join paper as t1 on t2.venueid  =  t1.venueid where t2.venuename  =  "chi";
Result: {'columns': ['distinct t1.paperid'], 'table': 'venue', 'where': 't2.venuename  =  "chi";'}


---------

In [9]:
file_path = "/Users/vedanttibrewal/Documents/USC/lectures/sem_1/DSCI-551/project/chatDB-dsci551/dataset/"
train_data = pd.read_csv(f"{file_path}train.csv", index_col=0)
train_data.head()

/var/folders/3w/z9ryb6gj05n_x375lp0r_81r0000gn/T/ipykernel_15009/1522993002.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  train_data = pd.read_csv(f"{file_path}train.csv", index_col=0)


,question,query,db_id,from,select,where,groupBy,having,orderBy,limit,...,union,except,distinct,count,max,avg,min,sum,and,or
0,Tell me what the notes are for South Australia,select notes from table where current slogan =...,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,What is the current series where the new serie...,select current series from table where notes =...,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,What is the format for South Australia?,select format from table where state/territory...,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Name the background colour for the Australian ...,select text/background colour from table where...,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,how many times is the fuel propulsion is cng?,select count fleet series ( quantity ) from ta...,0,1,1,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [62]:
# without stop words
temp_df = pd.DataFrame()
temp_df['processed'] = train_data['question'].map(preprocess_text)
temp_df['question'] = train_data['question']

--------

## Pattern Matching

### Table & Column identification

In [ ]:
# optional
def indentify_table(tokens, schema):
    for i, token in enumerate(tokens):
        if token.lower() == "from":
            table_name = tokens[i + 1]
            if table_name in schema:
                print(f"Table identified: {table_name}")
                return table_name
    

# implement fuzzy logic
def indentify_col_tables(tokens, schema):
    res = dict()
    for token in tokens:
        for table, columns in schema.items():
            if token in columns: # replace by fuzzy logic
                print(f"Column identified: {token} in table {table}")
                if res[table]:
                    res[table].append(token)
                else:
                    res[table] = list(token)

    return res





### Join identification

In [80]:
import networkx as nx

def create_graph(db_schema):
# Create an undirected graph from the SQL structure
    G = nx.Graph()  # This creates an undirected graph

# Add nodes and edges based on the SQL dictionary
    for table, details in db_schema.items():
        G.add_node(table)  # Add table as a node
        for fk_table in details['fk']:  # Iterate through foreign keys
            G.add_edge(table, fk_table)  # Create an undirected edge between current table and foreign key table

    # Get nodes and edges for verification
    nodes = list(G.nodes)
    edges = list(G.edges)

    # Output nodes and edges
    print("Nodes:", nodes)
    print("Edges:", edges)

    return G

# # Verify that the graph is undirected
# print("\nVerifying that the graph is undirected:")
# for edge in edges:
#     reverse_edge = (edge[1], edge[0])
#     print(f"Edge {edge} exists: {G.has_edge(*edge)}")
#     print(f"Reverse edge {reverse_edge} exists: {G.has_edge(*reverse_edge)}")
#     print()

G = create_graph(SQL)

# # Additional verification
# print(f"Is the graph directed? {G.is_directed()}")

Nodes: ['products', 'categories', 'customers', 'orders', 'order_items']
Edges: [('products', 'categories'), ('products', 'order_items'), ('customers', 'orders'), ('orders', 'order_items')]


In [86]:
def create_dir_graph(db_schema):
    G = nx.DiGraph()

    # Add nodes and edges based on the SQL dictionary
    for table, details in db_schema.items():
        G.add_node(table)  # Add table as a node
        for fk_table in details['fk']:  # Iterate through foreign keys
            G.add_edge(table, fk_table)  # Create an edge from current table to foreign key table

    # Get nodes and edges for verification
    nodes = list(G.nodes)
    edges = list(G.edges)

    # Output nodes and edges
    print(nodes)
    print(edges)
    return G

In [96]:
dir_g = create_dir_graph(db_schema=SQL)

['products', 'categories', 'customers', 'orders', 'order_items']
[('products', 'categories'), ('orders', 'customers'), ('order_items', 'orders'), ('order_items', 'products')]


In [ ]:
"""Undirected Graph"""

# def required_tables_graph(G, start, end, required_tables):
#     def dijkstra(graph, start, end):
#         distances = {node: float('infinity') for node in graph}
#         distances[start] = 0
#         pq = [(0, start)]
#         previous = {node: None for node in graph}

#         while pq:
#             current_distance, current_node = heapq.heappop(pq)

#             if current_node == end:
#                 path = []
#                 while current_node:
#                     path.append(current_node)
#                     current_node = previous[current_node]
#                 return path[::-1], current_distance

#             for neighbor in graph[current_node]:
#                 distance = current_distance + 1  # All edges have weight of 1
#                 if distance < distances[neighbor]:
#                     distances[neighbor] = distance
#                     previous[neighbor] = current_node
#                     heapq.heappush(pq, (distance, neighbor))

#         return None, float('infinity')

#     required_tables = set(required_tables) - {start, end}
#     best_path = None
#     best_distance = float('infinity')

#     def dfs(current_path, current_distance, remaining_required):
#         nonlocal best_path, best_distance

#         if not remaining_required:
#             path, distance = dijkstra(G, current_path[-1], end)
#             if path:
#                 total_path = current_path + path[1:]
#                 total_distance = current_distance + distance
#                 if total_distance < best_distance:
#                     best_path = total_path
#                     best_distance = total_distance
#             return

#         for node in remaining_required:
#             path, distance = dijkstra(G, current_path[-1], node)
#             if path:
#                 new_path = current_path + path[1:]
#                 new_distance = current_distance + distance
#                 new_remaining = remaining_required - {node}
#                 dfs(new_path, new_distance, new_remaining)

#     dfs([start], 0, required_tables)

#     return best_path, best_distance

# # # Set start, end, and required nodes
# # required_tables = ['categories', 'products', 'orders']
# # start = required_tables[1]
# # end = required_tables[1]

# # # Find the shortest path
# # required_tables, distance = required_tables_graph(G, start, end, required_tables)

# # print(f"Shortest path: {' -> '.join(required_tables) if required_tables else 'No path found'}")
# # print(f"Total distance: {distance}")

In [98]:
def required_tables_graph(G, start, end, required_tables):
    # Dijkstra's algorithm for shortest path in NetworkX graph
    def dijkstra(graph, start, end):
        distances = {node: float('infinity') for node in graph.nodes}
        distances[start] = 0
        pq = [(0, start)]
        previous = {node: None for node in graph.nodes}

        while pq:
            current_distance, current_node = heapq.heappop(pq)

            if current_node == end:
                path = []
                while current_node:
                    path.append(current_node)
                    current_node = previous[current_node]
                return path[::-1], current_distance

            for neighbor in graph.neighbors(current_node):  # Use NetworkX's neighbors method
                distance = current_distance + 1  # All edges have weight of 1
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous[neighbor] = current_node
                    heapq.heappush(pq, (distance, neighbor))

        return None, float('infinity')

    required_tables = set(required_tables) - {start, end}
    best_path = None
    best_distance = float('infinity')

    # Depth-first search (DFS) to explore all paths through required tables
    def dfs(current_path, current_distance, remaining_required):
        nonlocal best_path, best_distance

        if not remaining_required:
            path, distance = dijkstra(G, current_path[-1], end)
            if path:
                total_path = current_path + path[1:]
                total_distance = current_distance + distance
                if total_distance < best_distance:
                    best_path = total_path
                    best_distance = total_distance
            return

        for node in remaining_required:
            path, distance = dijkstra(G, current_path[-1], node)
            if path:
                new_path = current_path + path[1:]
                new_distance = current_distance + distance
                new_remaining = remaining_required - {node}
                dfs(new_path, new_distance, new_remaining)

    # Start DFS from the start node with an empty path and zero distance
    dfs([start], 0, required_tables)
    print(best_path)
    print(best_distance)

    return best_path, best_distance

In [104]:
print(dir_g.edges)

[('products', 'categories'), ('orders', 'customers'), ('order_items', 'orders'), ('order_items', 'products')]


In [107]:
required_tables_graph(dir_g, "categories", "orders", ["categories", "products", "orders"])

start_node = 'products'
end_node = ''
required_tables_list = ['categories', 'products', 'orders']

# Find the shortest path through required tables using the modified function
path_result, total_distance_result = required_tables_graph(dir_g, start_node, end_node, required_tables_list)


None
inf
None
inf


In [93]:
from ordered_set import OrderedSet

# refine logic
def join_clause(req_schema: dict, db_schema: dict):
    clause = ""
    max_cols = 0
    prim_table = None
    # change logic based on foreign key numbers
    for table, cols in req_schema.items():
        if len(cols) > max_cols:
            max_cols = len(cols)
            prim_table = table
    
    # pk_col = db_schema[prim_table]['pk']
    print(prim_table)

    # create graph from Database schema
    db_graph = create_dir_graph(db_schema)

    print(db_graph)

    required_tables = list(req_schema.keys())

    print(required_tables)

    min_dist = float('inf')
    for st_table in required_tables:
        for end_table in required_tables:
            sub_graph, distance = required_tables_graph(db_graph, st_table, end_table, required_tables)
            if distance < min_dist:
                join_graph = OrderedSet(sub_graph)
                min_dist = distance

    print(join_graph)

    req_graph = []

    for edge in db_graph.edges:
        # assumption binary relations between tables
        if edge[0] in join_graph and edge[1] in join_graph:
            req_graph.append(edge)

    print("required graph: ", req_graph)

    for i, edge in enumerate(req_graph):
        table1 = edge[0]
        table2 = edge[1]
        print(edge)
        if table1 in db_schema[table2]['fk']:
            # print("1")
            fk_col = db_schema[table2]['fk'][table1] # foreign key corresponding to 2nd table  primary key
            pk_col = db_schema[table1]['pk'][0] # primary key of 2nd table
            clause += f"JOIN {table1} ON {table2}.{fk_col}={table1}.{pk_col} \n"
        else:
            fk_col = db_schema[table1]['fk'][table] # foreign key corresponding to 2nd table  primary key
            pk_col = db_schema[table2]['pk'][0] # primary key of 2nd table
            clause += f"JOIN {table2} ON {table1}.{fk_col}={table2}.{pk_col} \n"
        
        # print("test: ", clause)

    # for i, table in enumerate(join_graph):
    #     if not db_schema[table]['fk']:
    #         continue
        
    # print(clause)

    # for table in req_schema.keys():
    #     if table != prim_table:
    #         fk_col = db_schema[prim_table]['fk'][table] # foreign key corresponding to 2nd table  primary key
    #         pk_col = db_schema[table]['pk'][0] # primary key of 2nd table
    #         clause += f"JOIN {table} ON {prim_table}.{fk_col}={table}.{pk_col} \n"
    #         print(clause)
    return clause

# create a graph out of db schema and select required tables starting with primary table (using shortest route)

In [95]:
print(join_clause(req, SQL))

customers
['products', 'categories', 'customers', 'orders', 'order_items']
[('products', 'categories'), ('orders', 'customers'), ('order_items', 'orders'), ('order_items', 'products')]
DiGraph with 5 nodes and 4 edges
['products', 'customers', 'orders']


UnboundLocalError: cannot access local variable 'join_graph' where it is not associated with a value

In [79]:
print(join_clause(req, SQL))

customers
Nodes: ['products', 'categories', 'customers', 'orders', 'order_items']
Edges: [('products', 'categories'), ('products', 'order_items'), ('customers', 'orders'), ('orders', 'order_items')]
Graph with 5 nodes and 4 edges
['products', 'customers', 'orders']
OrderedSet(['products', 'order_items', 'orders', 'customers'])
required graph:  [('products', 'order_items'), ('customers', 'orders'), ('orders', 'order_items')]
('products', 'order_items')
('customers', 'orders')
('orders', 'order_items')
JOIN products ON order_items.product_id=products.product_id 
JOIN customers ON orders.customer_id=customers.customer_id 
JOIN orders ON order_items.order_id=orders.order_id 



In [110]:
def find_path(graph, start, end, visited):
    if start == end:
        return [start]
    for neighbor in graph[start]:
        if neighbor not in visited:
            path = find_path(graph, neighbor, end, visited | {neighbor})
            if path:
                return [start] + path
    return None

def get_join_condition(sql, from_table, to_table):
    for column, fk_info in sql[from_table].get('fk', {}).items():
        if fk_info.split('.')[0] == to_table:
            return f'{from_table}.{column} = {fk_info}'
    for column, fk_info in sql[to_table].get('fk', {}).items():
        if fk_info.split('.')[0] == from_table:
            return f'{to_table}.{column} = {fk_info}'
    return None

def generate_sql_join(sql, req):
    join_statements = []
    tables = list(req.keys())
    
    # Start with the base table (assuming it's the first table in req)
    base_table = tables[0]
    join_statements.append(f'FROM {base_table}')
    
    # Create a graph of table relationships
    graph = {table: set() for table in sql.keys()}
    for table, info in sql.items():
        for fk_table, fk_column in info.get('fk', {}).items():
            graph[table].add(fk_table)
            graph[fk_table].add(table)
    
    # Find the join path
    visited = set([base_table])
    for table in tables[1:]:
        path = find_path(graph, base_table, table, visited)
        if path:
            for i in range(len(path) - 1):
                from_table, to_table = path[i], path[i+1]
                join_condition = get_join_condition(sql, from_table, to_table)
                join_statements.append(f'JOIN {to_table} ON {join_condition}')
                visited.add(to_table)
    
    return '\n'.join(join_statements)


In [111]:
generate_sql_join(SQL, req)

'FROM products\nJOIN order_items ON None\nJOIN orders ON None\nJOIN customers ON None'

In [108]:
"""FROM order_items
JOIN products on order_items.product_id=products.product_id
JOIN orders on orders.order_id = order_items.order_id
JOIN customers on order.customer_id = customer.customer_id
"""

'FROM order_items'

In [ ]:
"""SQL = {
    "books": {
        "pk": ["book_id"],
        "fk": {"authors": "author_id"},
        "book_id": "INT",
        "title": "VARCHAR(255)",
        "author_id": "INT",
        "isbn": "VARCHAR(13)",
        "publication_year": "INT",
        "price": "DECIMAL(10, 2)",
        "stock_quantity": "INT"
    },
    "authors": {
        "pk": ["author_id"],
        "fk": {},
        "author_id": "INT",
        "first_name": "VARCHAR(50)",
        "last_name": "VARCHAR(50)",
        "birth_date": "DATE",
        "nationality": "VARCHAR(50)",
        "biography": "TEXT"
    }
}

req = {
    "books": [
        "book_id",
        "title",
        "stock_quantity"
    ],
    "authors": [
        "author_id",
        "first_name",
        "last_name",
    ]
}"""

"""
SQL = {
    "employees": {
        "pk": ["employee_id"],
        "fk": {"departments": "department_id"},
        "employee_id": "INT",
        "first_name": "VARCHAR(50)",
        "last_name": "VARCHAR(50)",
        "email": "VARCHAR(100)",
        "phone_number": "VARCHAR(20)",
        "hire_date": "DATE",
        "job_title": "VARCHAR(100)",
        "salary": "DECIMAL(10, 2)",
        "department_id": "INT"
    },
    "departments": {
        "pk": ["department_id"],
        "fk": {"employees": "manager_id"},
        "department_id": "INT",
        "department_name": "VARCHAR(100)",
        "manager_id": "INT",
        "location": "VARCHAR(100)"
    },
    "job_history": {
        "pk": ["employee_id", "start_date"],
        "fk": {
            "employees": "employee_id",
            "departments": "department_id"
        },
        "employee_id": "INT",
        "start_date": "DATE",
        "end_date": "DATE",
        "job_title": "VARCHAR(100)",
        "department_id": "INT"
    }
}

req = {
    "employees": [
        "employee_id",
        "first_name",
        "last_name",
        "job_title"
    ],
    "departments": [
        "department_id",
        "department_name",
        "location"
    ]
}
"""
SQL = {
    "products": {
        "pk": ["product_id"],
        "fk": {"categories": "category_id"},
        "product_id": "INT",
        "product_name": "VARCHAR(255)",
        "description": "TEXT",
        "price": "DECIMAL(10, 2)",
        "stock_quantity": "INT",
        "category_id": "INT"
    },
    "categories": {
        "pk": ["category_id"],
        "fk": {},
        "category_id": "INT",
        "category_name": "VARCHAR(100)",
        "description": "TEXT"
    },
    "customers": {
        "pk": ["customer_id"],
        "fk": {},
        "customer_id": "INT",
        "first_name": "VARCHAR(50)",
        "last_name": "VARCHAR(50)",
        "email": "VARCHAR(100)",
        "phone_number": "VARCHAR(20)",
        "address": "TEXT"
    },
    "orders": {
        "pk": ["order_id"],
        "fk": {
            "customers": "customer_id"
        },
        "order_id": "INT",
        "customer_id": "INT",
        "order_date": "DATE",
        "total_amount": "DECIMAL(10, 2)",
        "status": "VARCHAR(20)"
    },
    "order_items": {
        "pk": ["order_item_id"],
        "fk": {
            "orders": "order_id",
            "products": "product_id"
        },
        "order_item_id":"INT",
        "order_id": "INT",
        "product_id": "INT",
        "quantity": "INT",
        "unit_price": "DECIMAL(10, 2)"
    }
}

req = {
    "products": [
        "product_id",
        "product_name",
        "price"
    ],
    "customers": [
        "customer_id",
        "first_name",
        "last_name",
        "email"
    ],
    "orders": [
        "order_id",
        "customer_id",
        "order_date",
        "total_amount"
    ]
}

In [68]:
join_clause(req, SQL)

customers
Nodes: ['products', 'categories', 'customers', 'orders', 'order_items']
Edges: [('products', 'categories'), ('products', 'order_items'), ('customers', 'orders'), ('orders', 'order_items')]
['products', 'customers', 'orders']
OrderedSet(['products', 'order_items', 'orders', 'customers'])
required graph:  [('products', 'order_items'), ('customers', 'orders'), ('orders', 'order_items')]


KeyError: 'orders'

### Condiiton identification 

In [ ]:
# import re

# difference between where and having
def identify_condition(tokens):
    conditions = []
    for i, token in enumerate(tokens):
        if token.lower() == "where":
            conditions.append(" ".join(tokens[i+1:])) # refine with 
            break

    print(f"Conditions: {conditions}")
    # Example Output: Conditions: ["age > 30"]


### Group By identification